## Data load

In [1]:
import os

if not os.path.exists('data/transactions_train.csv'):
    ! mkdir -p data
    ! curl -OL https://storage.yandexcloud.net/di-datasets/age-prediction-nti-sbebank-2019.zip
    ! unzip -j -o age-prediction-nti-sbebank-2019.zip 'data/*.csv' -d data
    ! mv age-prediction-nti-sbebank-2019.zip data/

## Setup

In [2]:
%load_ext autoreload
%autoreload 2

import logging
import pytorch_lightning as pl
import warnings

warnings.filterwarnings('ignore')
logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)

## Data preproccessing

In [4]:
import os
import pandas as pd

data_path = 'data/'

source_data = pd.read_csv(os.path.join(data_path, 'transactions_train.csv'))
source_data.head(2)

,client_id,trans_date,small_group,amount_rur
0,33172,6,4,71.463
1,33172,6,35,45.017


In [5]:
from ptls.data_preprocessing import PandasDataPreprocessor

preprocessor = PandasDataPreprocessor(
    col_id='client_id',
    cols_event_time='trans_date',
    time_transformation='float',
    cols_category=["trans_date", "small_group"],
    cols_log_norm=["amount_rur"],
    print_dataset_info=False,
)

In [6]:
%%time

dataset = preprocessor.fit_transform(source_data)

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.
CPU times: user 1min 8s, sys: 14.2 s, total: 1min 22s
Wall time: 1min 22s


In [7]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(dataset, test_size=0.2, random_state=42)

print(len(train), len(test))

24000 6000


## Embedding training

Model training in our framework organised via pytorch-lightning (pl) framework.
The key parts of neural networks training in pl are: 

    * model (pl.LightningModule)
    * data_module (pl.LightningDataModule)
    * pl.trainer (pl.trainer)
    
For futher details check https://www.pytorchlightning.ai/

### Model definition

In [8]:
from ptls.seq_encoder import SequenceEncoder
from ptls.models import Head
from ptls.lightning_modules.emb_module import EmbModule

seq_encoder = SequenceEncoder(
    category_features=preprocessor.get_category_sizes(),
    numeric_features=["amount_rur"],
    trx_embedding_noize=0.003
)

head = Head(input_size=seq_encoder.embedding_size, use_norm_encoder=True)

model = EmbModule(seq_encoder=seq_encoder, head=head)

### Data module

In [9]:
from ptls.data_load.data_module.emb_data_module import EmbeddingTrainDataModule

dm = EmbeddingTrainDataModule(
    dataset=train,
    pl_module=model,
    min_seq_len=25,
    seq_split_strategy='SampleSlices',
    category_names = model.seq_encoder.category_names,
    category_max_size = model.seq_encoder.category_max_size,
    split_count=5,
    split_cnt_min=25,
    split_cnt_max=200,
    train_num_workers=16,
    train_batch_size=256,
    valid_num_workers=16,
    valid_batch_size=256
)

### Trainer

In [10]:
import torch
import pytorch_lightning as pl

import logging
# logging.getLogger("lightning").addHandler(logging.NullHandler())
# logging.getLogger("lightning").propagate = False

trainer = pl.Trainer(
#     progress_bar_refresh_rate=0,
    max_epochs=150,
    gpus=1 if torch.cuda.is_available() else 0
)

### Training 

In [29]:
%%time

trainer.fit(model, dm)

## Inference 

In [28]:
# embedding inference

from ptls.inference import get_embeddings

train_embeds = get_embeddings(
    data=train,
    model=model, 
    category_names = model.seq_encoder.category_names,
    category_max_size = model.seq_encoder.category_max_size,
)

test_embeds = get_embeddings(
    data=test,
    model=model, 
    category_names = model.seq_encoder.category_names,
    category_max_size = model.seq_encoder.category_max_size,
)

train_embeds.shape, test_embeds.shape

In [15]:
# join target and embeddings

df_target = pd.read_csv(os.path.join(data_path, 'train_target.csv'))
df_target = df_target.set_index('client_id')
df_target.rename(columns={"bins": "target"}, inplace=True)

train_df = pd.DataFrame(data=train_embeds, columns=[f'embed_{i}' for i in range(train_embeds.shape[1])])
train_df['client_id'] = [x['client_id'] for x in train]
train_df = train_df.merge(df_target, how='left', on='client_id')

test_df = pd.DataFrame(data=test_embeds, columns=[f'embed_{i}' for i in range(test_embeds.shape[1])])
test_df['client_id'] = [x['client_id'] for x in test]
test_df = test_df.merge(df_target, how='left', on='client_id')

print(train_df.shape, test_df.shape)
train_df.head(2)

(24000, 514) (6000, 514)


,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,embed_7,embed_8,embed_9,...,embed_504,embed_505,embed_506,embed_507,embed_508,embed_509,embed_510,embed_511,client_id,target
0,0.392583,-0.294379,0.092941,-0.237422,0.404464,0.261568,-0.387329,0.152336,-0.171187,0.225108,...,0.629309,-0.384692,0.234163,0.157350,0.275607,0.039914,-0.487853,-0.334748,36253,1
1,0.398136,-0.217370,0.277116,-0.166076,0.429496,0.251350,-0.367074,0.279628,-0.256542,0.165144,...,0.468280,-0.483548,0.226620,0.055548,0.297315,0.211119,-0.060683,-0.403386,396,2


In [17]:
import numpy as np


y_train = train_df['target'].values
X_train = train_df.drop(['client_id', 'target'], axis=1)
X_train_emb = pd.DataFrame(np.arange(len(X_train)))
X_train_emb['embeddings'] = X_train.values.tolist()
X_train_emb = X_train_emb.drop([0], axis=1)

y_val = test_df['target'].values
X_val = test_df.drop(['client_id', 'target'], axis=1)
X_val_emb = pd.DataFrame(np.arange(len(X_val)))
X_val_emb['embeddings'] = X_val.values.tolist()
X_val_emb = X_val_emb.drop([0], axis=1)

## Lets use obtained embeddings for CatBoost training

In [18]:
from catboost import CatBoostClassifier, metrics

### With CatBoost embedding_features

In [19]:
CatBoostModel_emb = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.05,
    use_best_model=True,
    custom_metric=[metrics.Accuracy()],
    random_seed=42,
    logging_level='Silent',
    embedding_features=['embeddings'],
    depth=5
)

In [27]:
%%time

CatBoostModel_emb.fit(
    X_train_emb, y_train,
    eval_set=(X_val_emb, y_val),
    plot=True
#     logging_level='Verbose',  # you can uncomment this for text output
)

In [159]:
CatBoostModel_emb.get_best_score()

{'learn': {'Accuracy': 0.653, 'MultiClass': 0.8087743073729691},
 'validation': {'Accuracy': 0.6185, 'MultiClass': 0.8762496658268969}}

### Without CatBoost embedding_features

In [22]:
CatBoostModel = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.05,
    use_best_model=True,
    custom_metric=[metrics.Accuracy()],
    random_seed=42,
    logging_level='Silent',
    depth=4
)

In [26]:
%%time

CatBoostModel.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    plot=True
#     logging_level='Verbose',  # you can uncomment this for text output
)

In [204]:
CatBoostModel.get_best_score()

{'learn': {'Accuracy': 0.6882083333333333, 'MultiClass': 0.7527508410689302},
 'validation': {'Accuracy': 0.6341666666666667,
  'MultiClass': 0.8445338997736155}}

In [205]:
CatBoostModel.score(X_val, y_val)

0.6328333333333334